<center><h1>Eléments logiciels de traitement des données massives</h1></center>

<hr />
<center><h2>L'algorithme LambdaMART</h2></center>
<p style="text-align:right;">Johanna Lalou<br />Antoine Franz</p>
<hr />

Dans le cadre de ce projet, nous avons décidé de travailler sur une problématique de *Machine Learned Ranking*. Le Machine-Learned Ranking (MLR) ou Learning-To-Rank (LTR) correspond à une classe de techniques relativement récente dans laquelle des algorithmes de Machine Learning supervisés sont utilisés pour apprendre une fonction de ranking. C’est en particulier très important pour les moteurs de recherche de choisir une fonction de ranking précise car elles affectent directement l’expérience de recherche de millions d’utilisateurs. Ainsi, le MLR transforme un problème de recherche en un problème de Machine Learning. Il s’intéresse à une liste de documents, et le but est alors d’obtenir un classement optimal. Le MLR ne se concentre donc pas tant sur les scores exacts de chaque document, mais plutôt sur l’ordre relatif dans lequel il faut afficher ces documents. Ainsi, bien que l’application la plus commune du MLR soit le classement dans les moteurs de recherche, il peut être utilisé n’importe où où l’on a besoin de produire une liste ordonnée d’objets.

La base d’entraînement pour un tel modèle est constituée d’une liste de documents ainsi que d’un score pour chacun de ces documents. Pour un moteur de recherche, cela correspond à une liste de résultats associés à une requête, ainsi qu’à une note de pertinence de chacun des document associé à la requête. La plus répandue des façons d’obtenir ces notes de pertinence est de demander à des humains experts de noter manuellement des résultats en fonction de requêtes.


L'algorithme que nous avons choisi d'implémenter et de paralléliser est l'algorithme *LambdaMART*. Avant de présenter la technique de parallélisation dans l'environnement PyCuda et le choix du GPU, nous allons présenter l'algorithme.

La majorité des résultats présentés ici s'appuient sur le papier de Chris J.C. Burges, *From RankNet to LambdaRank to LambdaMART: An Overview* (June 23, 2010), disponible <a href="https://www.microsoft.com/en-us/research/publication/from-ranknet-to-lambdarank-to-lambdamart-an-overview/">ici</a>.


# I] L'algorithme LambdaMart [Théorie]

Avant d’aller plus loin dans la description théorique et dans les applications techniques d’un tel algorithme, tâchons de donner un premier aperçu général. Pour commencer, nous ne pouvons évoquer LambdaMart sans parler de ses proches prédécesseurs, RankNet et LambdaRank. RankNet, LambdaRank et LambdaMart sont des algorithmes de LTR développés par Chris Burges et ses partenaires, au centre de recherche de Microsoft. RankNet fut le premier développé, suivi par LambdaRank puis LambdaMart. Dans ces trois techniques, la notion de classement est transformée en un problème de classification ou régression par paire. Cela signifie que nous nous concentrons sur une paire de documents à la fois, nous trouvons le classement optimal pour cette paire, puis nous l’utilisons pour obtenir le classement final de tous les résultats.


##### a) RankNet

** Introduction à RankNet **

RankNet fut initiallement développé en utilisant des réseaux de neurones, mais le modèle sous-jacent ne se réduit pas à cette utilisation. Dans ce modèle, une fonction de coût est introduite, et le but est de minimiser le nombre de permutations dans le classement. Ici, une permutation signifie un ordre incorrect concernant une paire de résultats, c’est à dire quand un résultat mal noté est mieux classé qu’un résultat bien noté. Le modèle RankNet optimise la fonction de coût en utilisant une méthode de descente de gradients stochastiques (explicitée en partie 2.3).
Pour être plus précis, pour chaque requête et pour chaque paire de documents à classer pour cette requête, RankNet prend en compte les caractéristiques des deux documents et ajuste ses paramètres de proche en proche. Ainsi, les documents s’ajustent peu à peu en se "décalant" les uns par rapport aux autres. 

** Modèle théorique **

D'un point de vue plus formel, nous prenons des documents $x=(x_1,x_2,...,x_d)$ possédant des caractéristiques $x_i$, et notons $l_i$ les labels associés à chaque document (note de 0 à 4) pour une requête $q$ donnée.

Soit une fonction de classement $f$
\begin{align*}
 f: &\mathbb{R}^d \rightarrow  \mathbb{R} \\
& x=(x_1,x_2, ..., x_d) \rightarrow f(x)
\end{align*}

On définit $s_i=f(x_i)$. L'Idée clé de Ranknet est de se concentrer sur des paires d'éléments et de définir une probabilité particulière : Nous modélisons la probabilité de l’événement $U_i>U_j$ (document i plus pertinent que le document j) à l’aide d’une fonction sigmoïde, où $\sigma$ détermine la forme de cette dernière :
Plus $s_i-s_j$ est élevé (souvent appelée la marge), plus est grande la probabilité.


$$P_{ij}=P(U_i>U_j)=\frac{1}{1+exp(-\sigma(s_i-s_j))}$$

Nous pouvons définir la fonction d’entropie croisée (fonction de coût), qui mesure l’éloignement des probabilités
obtenues aux probabilités désirées : $C= - \hat{P_{ij}} log P_{ij} - (1-\hat{P_{ij}})log(1-P_{ij})$, $\hat{P_{ij}}=\frac{1+S_{ij}}{2}$ ($S_{ij}$ valant 1 si le document i est plus pertinent que le document j pour cette requête). En minimisant cette dernière, nous rendons la distribution de probabilité apprise par f au plus près que possible de la distribution
empirique de probabilité induite par les paires de jugement.  

Nous utilisons alors une méthode de descente des gradients stochastique. Le gradient est utilisé pour mettre à jour les paramètres $w_k$ (c’est à dire les paramètres du modèle), en réduisant le coût. L’algorithme du gradient stochastique est une méthode de descente de gradient (itérative) utilisée pour la minimisation d’une fonction objectif qui est écrite comme une somme de fonctions différentiables. A chaque étape, cette méthode tire un échantillon aléatoire de l’ensemble des fonctions constituant la somme. Cette astuce devient très efficace dans le cas de problème d’apprentissage à grande ou très grande échelle.

 $$w_k \rightarrow w_k-\eta\frac{\partial C}{\partial w_k} $$
 
 <img src="ranknet.png" width=400/>

Enfin, on définit $I=\{(i,j) / S_{ij}=1$\} et on obtient deux expressions fondamentales (calculs non détaillés ici) :

$\lambda_{ij}=\frac{\partial C(s_i-s_j)}{\partial s_i}$ et $\lambda_i=\sum_{j/(i,j)\in I}\lambda_{ij}-\sum_{j/(j,i)\in I}\lambda_{ij}$

$\frac{\partial C}{\partial w_k}=\lambda_{ij}(\frac{\partial s_i}{\partial w_k}-\frac{\partial s_j}{\partial w_k})$,


Interprétation des lambdas :

Nous pouvons considérer les documents comme étant des points ayant une massse. $\lambda_i$ est alors la force résultante sur le point masse $i$ . Plus précisément : <br />
— La première somme représente l’ensemble des forces venant des documents moins pertinents, poussant vers le haut du  classement.<br />
— La seconde somme représente l’ensemble des forces venant des documents plus pertinents, poussant vers le bas du classement.<br />
— Durant l’entraînement, la magnitude des forces change.


##### b) LambdaRank

** Mesurer la performance des résultats **

Une autre différence majeure avec les problèmes classiques de Machine Learning vient aussi des quantités qui mesurent l'erreur. Il faut pouvoir dire si une recherche est bonne ou mauvaise (ou entre les deux). Les systèmes de recherche essaient d’approximer un ordre idéal associé à une requête. En d’autres termes, le but est de minimiser la distance d’une liste de résultats idéaux à une liste de résultats obtenus pour une recherche. Des métriques particulières ont été spécialement conçues pour mesurer cela. Elles sont toujours comprises entre 0 (pire des résultats possibles) et 1 (jeu de résultats parfait). De plus, ces métriques doivent prendre en considération le fait que les utilisateurs se concentrent en particulier sur les premiers résultats présentés. C’est pourquoi ces métriques incluent souvent un biais de position signifiant que si un résultat en haut de la liste dévie de l’idéal, la recherche est considérée comme plus mauvaise que si un document en bas de liste dévie de l’idéal.
De nombreuses mesures de la qualité du classement sont disponibles : MRR (Mean Reciprocal Rank), MAP (Mean Average Precision), ERR (Expected Reciprocal Rank) and NDCG (Normalized Discounted Cumulative Gain). Nous ne nous intéresserons ici seulement à la dernière (NDCG), celle-ci ayant l’avantage de traiter plusieurs niveaux de pertinence et d’inclure une dépendance en la position des résultats. Ainsi, en se donnant un jeu de résultats associé à une requête donnée, nous pouvons définir :
$$ DCG@k= \sum_{i=1}^{k} \frac{2^{l_i}-1}{log(i+1)}$$
$$ NDCG@k=\frac{DCG@k}{max DCG@k} $$


avec $k$ correspondant au nombre de documents à optimiser dans la liste de résultats, et $l_i$ correspondant à la note attribuée aux documents, allant de 0 à 4. Ce choix d’indicateur est naturel lorsque nous utilisons l’algorithme LambdaMart (cf parties suivantes). Etant donné le jeu de données test en entrée, celui-ci est groupé par requête et pour chaque requête le ranking idéal (obtenu en classant les documents par ordre décroissant de pertinence) est comparé au ranking général par le modèle. Une moyenne de toutes les requêtes est alors calculée.


** Introduction à LambdaRank **

Burges découvrit que durant la phase d’entraînement du modèle RankNet, il n’est pas nécessaire de calculer les coûts, mais seulement les gradients ($\lambda$) des coûts respectivement aux scores du modèle. Nous pouvons alors nous représenter les gradients comme de petites flèches attachées à chaque document dans la liste de résultats, indiquant la direction dans laquelle nous souhaitons que les documents évoluent. Plus tard, il découvrit que redimensioner les gradients par le changement en NDCG trouvé en échangeant chaque paire de document donnait de bons résultats. L’idée principale de LambdaRank est d’utiliser cette nouvelle fonction de coût pour entraîner un modèle RankNet. Sur des jeux de données expérimentaux, cela accéléra le traitement et améliora la précision de l’algorithme originel.
Pour résumer, LambdaRank prend simplement les gradients de RankNET, et les multiplie par le changement en NDCG pour chaque paire de document. Cela améliore la vitesse et la pertinence.

<img src="ranknet2.png" width=400/>


##### c) MART 

MART est un modèle de Gradient Tree Boosting dans lequel la sortie du modèle est une combinaison linéaire d’outputs d’un set d’arbres de régression. Le modèle final fait correspondre à un vecteur $x$ un score $F(x)$ dans $R$.
$F_N(x)= \sum_{i=1}^N \alpha_i f_i(x)$ où chaque $fi(x) \in R$ est une fonction modélisée par un simple arbre de régression, et où les $\alpha_i \in R$ correspondent aux poids associés au i-ème arbre de régression. Ces deux derniers paramètres sont appris pendant la phase d’entraînement.

De plus, chaque arbre peut se réécrire sous la forme : $f_i(x)=\sum_{k=1}^L \gamma_{ki}\mathbb{1}(x \in R_k)$.
 Pour séparer les données en deux sous-branches au sein d'un arbre, on résout : $ \min_{j,s}[\min_{c_1} \sum_{x_i \in R_1(j,s)} (y_i - c_1)^2 + \min_{c_2} \sum_{x_i \in R_2(j,s)} (y_i - c_2)^2]$ avec $R_1(j,s)=\{X|X_j \leq s \}$ and $R_2(j,s)=\{X|X_j > s \}$ 

Ainsi, lorsque n arbres ont été entraînés, comment entraîner le suivant ? MART utilise une descente des gradients pour réduire la perte. Plus précisément, la prochaine génération d’arbres modélise les dérivées du coût en respect avec le modèle courant. 
Plus précisément, l'arbre $n+1$ modélise les $m$ dérivées du coût : $\frac{\partial C(F_n)}{\partial F_n}(x_i), i=1...m $. Enfin, la valeur des feuilles est calculée à partir de l'approximation de Newton.




##### d) LambdaMART

Ainsi, puisque le modèle MART modélise les dérivées et puisque le modèle LambdaRank a besoin de ces dérivées, les deux algorithmes peuvent facilement se regrouper pour créer LambdaMart. 

<img src="LambdaMart.png" width=800 />

L'algorithme est le suivant :

<img src="AlgoLambda.png" width=600 />

# II] L'algorithme LambdaMart [Implémentation]

L'algorithme respecte les étapes présentées ci-dessus. Nous avons commenté le code afin que chaque étape soit la plus claire possible. La version parallélisée reprendra une partie de ce code.

In [1]:
import numpy as np
import math
import random
import copy
from sklearn.tree import DecisionTreeRegressor
import pandas as pd

# Retourne la liste des dcg en parcourant la liste des documensts 
def dcg(scores):
    return np.sum([(np.power(2, scores[i]) - 1) / np.log2(i + 2) for i in range(len(scores))]) 

# Retourne la liste des dcg en parcourant la liste des documensts 
#(et en tronquant, c'est-à dire en ne prenant que les k premiers documents)
def dcg_k(scores, k):
    return np.sum([(np.power(2, scores[i]) - 1) / np.log2(i + 2) for i in range(len(scores[:k]))])

# Retourne la liste des dcg en parcourant la liste des documensts dans l'odre décroissant des scores 
# Autrement dit, les documents sont bien classés
def ideal_dcg(scores):
    scores = [score for score in sorted(scores)[::-1]]
    return dcg(scores)

# Retourne la liste des dcg en parcourant la liste des documensts dans l'odre décroissant des scores
#(et en tronquant, c'est-à dire en ne prenant que les k premiers documents)
def ideal_dcg_k(scores, k):
    scores = [score for score in sorted(scores)[::-1]]
    return dcg_k(scores, k)

# Retourne le dcg de deux documents 
def single_dcg(scores, i, j):
    return (np.power(2, scores[i]) - 1) / np.log2(j + 2)

# Calule les lambdas (qui vont permettre la descente de gradient)
def compute_lambda(args):
     # On prend comme argument la liste des vrais scores, la liste des scores prédits, 
    # les paires de documents telles que le document i a un score plus élevé que le document j,
    # la liste des idcg des documents et la liste des requêtes  
    true_scores, predicted_scores, good_ij_pairs, idcg, query_key = args  
    # Compte le nombre de documents
    num_docs = len(true_scores)
    # Trie les places des documents selon l'ordre décroissant des scores
    # Autrement nous avons en premier la place dans la liste du document au plus haut score etc.
    sorted_indexes = np.argsort(predicted_scores)[::-1]
    # Trie les places des documents selon l'ordre croissant des scores
    # Autrement nous avons en premier la place dans la liste du document au plus bas score etc.
    rev_indexes = np.argsort(sorted_indexes)
    # Nous prenons les vrais scores des documents triés selon l'ordre décoissant des scores
    true_scores = true_scores[sorted_indexes]
    # Nous prenons les scores prédits des documents triés selon l'ordre décoissant des scores
    predicted_scores = predicted_scores[sorted_indexes]
    # Nous créons un vecteur vide de la taille le nombre de documents pour les lambdas
    lambdas = np.zeros(num_docs)
    # Nous créons un vecteur vide de la taille le nombre de documents pour les poids
    w = np.zeros(num_docs)
    # Nous créons un dictionnaire donc les clés sont les paires de documents et les valeurs les dcg entre les deux documents
    single_dcgs = {}
    # On parcourt les paires de documents donc le document i a un score supérieur au document j
    # On rajoute au dictinnaire les cdg entre i et j et entre j et i, puis entre i et i et j et j s'ils n'existent pas déjà
    for i,j in good_ij_pairs:
        if (i,i) not in single_dcgs:
            single_dcgs[(i,i)] = single_dcg(true_scores, i, i)
        single_dcgs[(i,j)] = single_dcg(true_scores, i, j)
        if (j,j) not in single_dcgs:
            single_dcgs[(j,j)] = single_dcg(true_scores, j, j)
        single_dcgs[(j,i)] = single_dcg(true_scores, j, i) 
    for i,j in good_ij_pairs:
        # Calcul des NDCG pour chaque paire (différences de dcg sur dcg de i (le plus grand dcg))
        z_ndcg = abs(single_dcgs[(i,j)] - single_dcgs[(i,i)] + single_dcgs[(j,i)] - single_dcgs[(j,j)]) / idcg
        # Fonction définie dans l'article pour le calcul des lambdas
        rho = 1 / (1 + np.exp(predicted_scores[i] - predicted_scores[j]))
        # rho_complement ?
        rho_complement = 1.0 - rho
        # On multiplie le NDCG pour la paire i,j par rho
        lambda_val = z_ndcg * rho
        # On fait la sommme des lambda i,j pour obtenir les lambda i et les lambda j
        lambdas[i] += lambda_val
        lambdas[j] -= lambda_val
        # On met à jour les poids aussi 
        w_val = rho * rho_complement * z_ndcg
        w[i] += w_val
        w[j] += w_val
    # On retourne les lambdas et les poids pour les documents triés dans l'ordre croissant des scores
    # Autrement dit, en premier, on a le document avec le plus faible score
    return lambdas[rev_indexes], w[rev_indexes], query_key

def group_queries(training_data, qid_index):
    # On crée un dictionnaire dont les clés sont les requêtes et le valeurs les listes des places des documents dans la liste
    # des documents qu'on utilise dans la base train ( liste de listes contenant la requête le score et les features associés aux documents)
    query_indexes = {}
    index = 0
    for record in training_data:
        query_indexes.setdefault(record[qid_index], [])
        query_indexes[record[qid_index]].append(index)
        index += 1
    return query_indexes
 
# En entrée de la fonction, on a une liste de listes de score selon la requête
def get_pairs(scores):
    #On définit une liste 
    query_pair = []
    # Ici, on parcourt les listes de la liste 
    for query_scores in scores:
        # On trie les scores dans l'ordre décroissant (plus grands scores en premeirs)
        temp = sorted(query_scores, reverse=True)
        pairs = []
        for i in range(len(temp)):
            for j in range(len(temp)):
                if temp[i] > temp[j]:
                    pairs.append((i,j))
        # On ajoute à la liste finale 
        query_pair.append(pairs)
    return query_pair

# On définit la classe LambdaMART
class LambdaMART:

    #On fixe 5 arbres et un learning rate de 0.1.. on pourra tester d'autres learning rate après!!
    def __init__(self, training_data=None, number_of_trees=5, learning_rate=0.1):

        self.training_data = training_data
        self.number_of_trees = number_of_trees
        self.learning_rate = learning_rate
        self.trees = []
    
    # La fonction fit permet de fitter l'arbre aux lambdas
    def fit(self):
        # On définit autant de scores prédits que de lignes dans le training set
        predicted_scores = np.zeros(len(self.training_data))
        # On définit notre dictionnaire dont les clés sont les requêtes et les valeurs les places des documents
        # dans la liste des documents  ( le 1 parce que la deuxième valeur de la liste pour chaque document a en deuxième place la requête)
        query_indexes = group_queries(self.training_data, 1)
        # On récupère la liste des requêtes
        query_keys = query_indexes.keys()
        #On obtient ici une liste de listes de scores, par requête
        true_scores = [self.training_data[query_indexes[query], 0] for query in query_keys]
        #On obtient ici les paires de documents pour chaque liste de scores, donc pour chaque requête
        good_ij_pairs = get_pairs(true_scores)
        #On obtient ici l'ensemble des vecteurs des features pour les documents
        tree_data = pd.DataFrame(self.training_data[:, 2:7])
        # On obtient ici l'ensemble des scores pour les documents (les labels)
        labels = self.training_data[:, 0]
        # On calcule la liste des dcg pour les listes des scores classés dans l'ordre décroissant (classement idéal)
        # On obtient une liste de listes encore
        idcg = [ideal_dcg(scores) for scores in true_scores]
        # On parcourt le nombre d'arbres 
        for k in range(self.number_of_trees):
            #On crée  autant de lambdas qu'on a de lignes dans notre training set
            lambdas = np.zeros(len(predicted_scores))
            #On crée  autant de poids qu'on a de lignes dans notre training set
            w = np.zeros(len(predicted_scores))
            #On obtient ici une liste de listes de scores prédits, par requête
            pred_scores = [predicted_scores[query_indexes[query]] for query in query_keys]
            # pool permet de parralléliser déjà sur les 4 coeurs de l'ordi, par requête 
            # Ici, on obtient les lambdas, les poids pour les documents de chaque requête
            # En entrée de compute_lambda, on a les listes des vrais scores pour les docs de chaque requête,
            # les listes des scores prédits, les listes des paires de docs tels que le premeier élément a un score plus élevé que le deuxième,
            # la liste des dcg du classement idéal des documents, par requête
            for lambda_val, w_val, query_key in map(compute_lambda, zip(true_scores, pred_scores, good_ij_pairs, idcg, query_keys)):
                # Ici, on obtient la liste des places des documents associés à la requête 
                indexes = query_indexes[query_key]
                # On obtient les lambdas des docs associés à la requête en entrée, qu'on entre dans le vecteur lambda final
                lambdas[indexes] = lambda_val
                # On obtient les poids des docs associés à la requête en entrée,  qu'on entre dans le vecteur  poids final
                w[indexes] = w_val
            # Implementation sklearn of the three
            tree = DecisionTreeRegressor(max_depth=50)
            # On fit l'arbre aux lambdas
            tree.fit(self.training_data[:,2:], lambdas)
            # On ajoute l'arbre à notre liste d'arbres
            self.trees.append(tree)
            # On prédit grâce à notre arbre et à nos features  (la fonction predict est en dessous)
            # Calcule la prédiction pour chaque document grâce à nos précédents arbres et au nouveau
            prediction = tree.predict(self.training_data[:,2:])
            # On met à jour nos prédictions à l'aide de notre prédiciton récente et du learning_rate
            predicted_scores += prediction * self.learning_rate

    
    def predict(self, data):
        data = np.array(data)
        query_indexes = group_queries(data, 0)
        predicted_scores = np.zeros(len(data))
        for query in query_indexes:
            results = np.zeros(len(query_indexes[query]))
            for tree in self.trees:
                results += self.learning_rate * tree.predict(data[query_indexes[query], 1:])
            predicted_scores[query_indexes[query]] = results
        return predicted_scores

    #Fonction validate qui prédit sur la base test selon les tous les arbres qu'on a construit, qui calcule aussi la moyenne des NDCG
    def validate(self, data, k):
        data = np.array(data)
        query_indexes = group_queries(data, 1)
        average_ndcg = []
        predicted_scores = np.zeros(len(data))
        for query in query_indexes:
            results = np.zeros(len(query_indexes[query]))
            for tree in self.trees:
                results += self.learning_rate * tree.predict(data[query_indexes[query], 2:])
            predicted_sorted_indexes = np.argsort(results)[::-1]
            t_results = data[query_indexes[query], 0]
            t_results = t_results[predicted_sorted_indexes]
            predicted_scores[query_indexes[query]] = results
            dcg_val = dcg_k(t_results, k)
            idcg_val = ideal_dcg_k(t_results, k)
            ndcg_val = (dcg_val / idcg_val)
            average_ndcg.append(ndcg_val)
        average_ndcg = np.nanmean(average_ndcg)
        return average_ndcg, predicted_scores



In [2]:
import numpy as np
import pandas as pd
import time


# Main fonction
def main():
    start_time = time.time()
    total_ndcg = 0.0
    df=pd.read_csv('train2.csv')
    values=[[df.loc[(df.index == j),][str(u)][j] for u in list(range(1,49))] for j in range(len(df.index))][0:18]
    training_data = np.asarray(values)
    df2=pd.read_csv('test2.csv')
    values2=[[df2.loc[(df2.index == j),][str(u)][j] for u in list(range(1,49))] for j in range(len(df2.index))][0:18]
    test_data = np.asarray(values2)
    # On construit les 100 arbres sur la base test
    model = LambdaMART(training_data, 100, 0.001)
    model.fit()
    # On tronque sur les 10 premiers docs pour les calculs de la moyenne des DCG
    # On calcule les scores prédits sur la base test
    average_ndcg, predicted_scores = model.validate(test_data, 10) 
    print (average_ndcg)
    print("--- %s seconds ---" % (time.time() - start_time))


    
if __name__ == '__main__':
    main()

0.79364325119
--- 13.477192163467407 seconds ---


# III] Parallélisation à l'aide de pyCuda

#### a) Mise en place d'un GPU sur AWS

Afin de nous placer dans des conditions réelles, nous avons mis en place un GPU sous Amazon Web Service et avons travaillé sur des Notebooks directement sur le GPU.

Nous avons pour cela lancé une nouvelle instance en Irlande (la disponiblité de serveurs en France est apparue après que nous ayions configuré l'instance), à l'aide de l'AMI "Deep Learning AMI" :

<img src="ami.png" width=700 />


Nous nous connectons alors à l'instance via SSH à l'aide de la commande suivante :

*ssh -i "GPULambdaMart.pem" ubuntu@ec2-34-241-166-204.eu-west-1.compute.amazonaws.com* (l'adresse IP change dès que nous stoppons puis relançons l'instance, le fichier *GPULambdaMart.pem* correspond à une clé fournise par Amazon au moment de la création de l'instance).

<img src="screen.png"  />

Le tutoriel nous ayant servi à la mise en place du GPU est disponible <a href"https://blog.keras.io/running-jupyter-notebooks-on-gpu-on-aws-a-starter-guide.html">ici</a>.


#### b) La découpe thread/bloc

Une grille correspond à un arrangement en deux dimensions de blocs indépendants. La dimension d'une grille est $gridDim.x*gridDim.y$. Les coordonnées d'un bloc au sein de la grille sont les suivantes : ($blockIdx.x$,$blockIdx.y$).
Un bloc, quand à lui, correspond à un arrangement en trois dimensions de "threads", unités traitant effectivement le calcul. Les dimensions du bloc sont alors ($blockDim.x*blockDim.y*blockDim.z$), et un thread est accessible via les coordonnées ($threadIdx.x, threadIdx.y, threadIdx.z$). Le schéma ci-dessous illustre ce principe.



<img src="threads.png",width=800>

L'identifiant 26 correspond alors à blockIdx.x * blockDim.x + threadIdx.x=$3*8+2$.

#### c) Parallélisation

Nous avons alors transposé le code précédent en parallélisant le calcul des lambdas sur chaque thread du GPU, à l'aide de pyCuda. Chaque thread reçoit toutes les informations dont il a besoin pour pouvoir effectuer ses calculs, et modifie un vecteur global lambda modifié par tous les calculs parallèles, à des positions différentes (aucune possibilité de conflit). Nous devons donc dans un premier temps envoyer toutes les variables, valeurs et vecteurs vers le GPU. Puis le GPU effectue ses calculs à l'aide des fonctions écrites en C (en rouge dans le code ci-dessous).  ***fill_lambdas*** (global) répartie les calculs dans les différents threads en envoyant seulement une partie des données globales, nécessaires au calcul des lambdas, à une seconde fonction ***compute_lambdas*** (de type device).


**La réécriture de l'algorithme en C correspond à la phase qui nous a pris le plus de temps dans la réalisation de ce projet (notamment car nous n'avions jamais codé dans un langage bas-niveau). Nous avons du réécrire une partie des opérations élémentaires en C (telle que la fonction argsort), contrôler et modifier les types (dictionnaire en python, liste à plusieurs niveaux, pointeurs de pointeurs etc...).**

Enfin, la paréllisation s'effectue au niveau des requêtes, et ne peut pas ***a priori*** s'effectuer au niveau des arbres entiers, car chaque arbre repose sur les résultats fournis par l'arbre précédent...

#### c) Implémentation (à exécuter sur le GPU)

In [ ]:
import numpy as np
import math
import random
from sklearn.tree import DecisionTreeRegressor
import pandas as pd
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule


# Retourne la liste des dcg en parcourant la liste des documensts 
def dcg(scores):
    return np.sum([(np.power(2, scores[i]) - 1) / np.log2(i + 2) for i in range(len(scores))]) 

# Retourne la liste des dcg en parcourant la liste des documensts 
#(et en tronquant, c'est-à dire en ne prenant que les k premiers documents)
def dcg_l(scores, l):
    return np.sum([(np.power(2, scores[i]) - 1) / np.log2(i + 2) for i in range(len(scores[:l]))])

# Retourne la liste des dcg en parcourant la liste des documensts dans l'odre décroissant des scores 
# Autrement dit, les documents sont bien classés
def ideal_dcg(scores):
    scores = [score for score in sorted(scores)[::-1]]
    return dcg(scores)

# Retourne la liste des dcg en parcourant la liste des documensts dans l'odre décroissant des scores
#(et en tronquant, c'est-à dire en ne prenant que les k premiers documents)
def ideal_dcg_l(scores, l):
    scores = [score for score in sorted(scores)[::-1]]
    return dcg_l(scores, l)

# Retourne le dcg de deux documents 
def single_dcg(scores, i, j):
    return (np.power(2, scores[i]) - 1) / np.log2(j + 2)


def group_queries(training_data, qid_index):
    # On crée un dictionnaire dont les clés sont les requêtes et le valeurs les listes des places des documents dans la liste
    # des documents qu'on utilise dans la base train ( liste de listes contenant la requête le score et les features associés aux documents)
    query_indexes = {}
    index = 0
    for record in training_data:
        query_indexes.setdefault(record[qid_index], [])
        query_indexes[record[qid_index]].append(index)
        index += 1
    return query_indexes
 
# En entrée de la fonction, on a une liste de listes de score selon la requête
def get_pairs(scores):
    #On définit une liste 
    query_pair = []
    # Ici, on parcourt les listes de la liste 
    for query_scores in scores:
        # On trie les scores dans l'ordre décroissant (plus grands scores en premeirs)
        temp = sorted(query_scores, reverse=True)
        pairs = []
        for i in range(len(temp)):
            for j in range(len(temp)):
                if temp[i] > temp[j]:
                    pairs.append((i,j))
        # On ajoute à la liste finale 
        query_pair.append(pairs)
    return query_pair



class LambdaMART:

    #On fixe 5 arbres et un learning rate de 0.1.. on pourra tester d'autres learning rate après!!
    def __init__(self, training_data=None, number_of_trees=5, learning_rate=0.1):
        self.training_data = training_data
        self.number_of_trees = number_of_trees
        self.learning_rate = learning_rate
        self.trees = []
    
    # La fonction fit permet de fitter l'arbre aux lambdas
    def fit(self):
        # On définit autant de scores prédits que de lignes dans le training set
        predicted_scores = np.zeros(len(self.training_data))
        # On définit notre dictionnaire dont les clés sont les requêtes et les valeurs les places des documents
        # dans la liste des documents  ( le 1 parce que la deuxième valeur de la liste pour chaque document a en deuxième place la requête)
        query_indexes = group_queries(self.training_data, 1)
        # On récupère la liste des requêtes
        query_keys = query_indexes.keys()
        #On obtient ici une liste de listes de scores, par requête
        true_scores = [self.training_data[query_indexes[query], 0] for query in query_keys]
        #On obtient ici les paires de documents pour chaque liste de scores, donc pour chaque requête
        pairs = get_pairs(true_scores)
        #On obtient ici l'ensemble des vecteurs des features pour les documents
        tree_data = pd.DataFrame(self.training_data[:, 2:7])
        # On obtient ici l'ensemble des scores pour les documents (les labels)
        labels = self.training_data[:, 0]
        # On calcule la liste des dcg pour les listes des scores classés dans l'ordre décroissant (classement idéal)
        # On obtient une liste de listes encore
        idcg = [ideal_dcg(scores) for scores in true_scores]
        # On parcourt le nombre d'arbres 
        for k in range(self.number_of_trees):
            
            #On crée  autant de lambdas qu'on a de lignes dans notre training set
            lambdaFinal = np.ones(len(predicted_scores))
            w = np.zeros(len(predicted_scores))
            pred_scores = [predicted_scores[query_indexes[query]] for query in query_keys]
            
            
            # ON ENVOIE LES DONNEES VERS LE GPU
            
            # On alloue la bonne taille dans la mémoire du GPU
            lambda_gpu = cuda.mem_alloc(lambdaFinal.nbytes)
            # Puis on prépare l'envoi de la variable vers le GPU ! (bien vérifier qu'elle est de type array, sinon la convertir)
            cuda.memcpy_htod(lambda_gpu, lambdaFinal)
            # Et ainsi de suite pour toutes les variables
        
            pred_scores=np.asarray(pred_scores)
            pred_scores_gpu=cuda.mem_alloc(pred_scores.nbytes)
            cuda.memcpy_htod(pred_scores_gpu, pred_scores)
            
            tuple1=[([i[j][0] for j in range(len(i))])  for i in pairs]
            tuple2=[([i[j][1]  for j in range(len(i))])  for i in pairs]
        
            true_scores=np.asarray(true_scores)
            true_scores_gpu = cuda.mem_alloc(true_scores.nbytes)
            cuda.memcpy_htod(true_scores_gpu, true_scores)

            predicted_scores_gpu = cuda.mem_alloc(predicted_scores.nbytes)
            cuda.memcpy_htod(predicted_scores_gpu, predicted_scores)

            tuple11=np.array(tuple1)
            tuple11_gpu = cuda.mem_alloc(tuple11.nbytes)
            cuda.memcpy_htod(tuple11_gpu, tuple11)
            tuple2=np.asarray(tuple2)
            tuple2_gpu = cuda.mem_alloc(tuple2.nbytes)
            cuda.memcpy_htod(tuple2_gpu, tuple2)

            idcg=np.asarray(idcg)

            idcg_gpu = cuda.mem_alloc(idcg.nbytes)
            cuda.memcpy_htod(idcg_gpu, idcg)

            
            query_keys2=[[int(key)] for key in list(query_keys)]
            query_keys2=np.array(query_keys2, )
           
            
            query_keys_gpu = cuda.mem_alloc(query_keys2.nbytes)
            cuda.memcpy_htod(query_keys_gpu, query_keys2)

            
            keys=list(query_indexes.keys())
            lengthValues=[len(query_indexes[key]) for key in keys]
            maxLengthValues=max(lengthValues)
            
            tab=[]

            for i in range(len(keys)):
                tabint=[]
                tabint.append(int(keys[i]))
                tabint.append(lengthValues[i])
               
                for k in range(2,lengthValues[i]+2):
                    tabint.append(query_indexes[keys[i]][k-2])
                for l in range(lengthValues[i]+2,maxLengthValues):
                    tabint.append(0)
                tab.append(tabint)
            
            tab=np.asarray(tab)
            
            
            tab_gpu = cuda.mem_alloc(tab.nbytes)
            cuda.memcpy_htod(tab_gpu, tab)
            
            nb=len(keys)
            nbKeys=[nb]
            nbKeys=np.asarray(nbKeys)
            nbKeys_gpu = cuda.mem_alloc(nbKeys.nbytes)
            cuda.memcpy_htod(nbKeys_gpu, nbKeys)
            
            maxLengthValues = np.int_(maxLengthValues)
            maxLengthValues_gpu = cuda.mem_alloc(maxLengthValues.nbytes)
            cuda.memcpy_htod(maxLengthValues_gpu, maxLengthValues)
        
            
            
            # FIN DE l'ENVOI DES DONNEES VERS LE GPU
          
            
            # DEBUT DE L'ECRITURE DES FONCTIONS EN C (qui seront exécutées sur le GPU). Ce code (notamment 
            # la fonction compute_lambda), fait la même chose que le code explicité dans la partie ci-dessus, en python,
            # mais cette fois-ci en C (avec les modifications d'objet et de syntaxe que cela implique)
            mod = SourceModule("""
        
            #include <stdio.h>
            #include <math.h>
            
            __device__ int comp(const void * elem1, const void * elem2) {
                int f = *((int*)elem1);
                int s = *((int*)elem2);
                if (f > s) return  1;
                if (f < s) return -1;
                return 0;
            }

            __device__ void swap(int *xp, int *yp)
            {
                int temp = *xp;
                *xp = *yp;
                *yp = temp;
            }


            __device__ void bubbleSort(int arr[], int n)
            {
               int i, j;
               for (i = 0; i < n-1; i++)      

                   // Last i elements are already in place   
                   for (j = 0; j < n-i-1; j++) 
                       if (arr[j] > arr[j+1])
                          swap(&arr[j], &arr[j+1]);
            }
            __device__ void swapDouble(double *xp, double *yp)
            {
                double temp = *xp;
                *xp = *yp;
                *yp = temp;
            }


            __device__ void bubbleSortDouble(double arr[], int n)
            {
               int i, j;
               for (i = 0; i < n-1; i++)      

                   // Last i elements are already in place   
                   for (j = 0; j < n-i-1; j++) 
                       if (arr[j] > arr[j+1])
                          swapDouble(&arr[j], &arr[j+1]);
            }



                 __device__ int compute_lambda(int *maxLengthValues, int *true_scores, double *predited_scores, int *tuple1, int *tuple2, double *idcg, int *query_key, int tab[][5], int *nbKeys, double *lambda){  



               int i = 0;
               int j = 0;
               int q = 0;
               int val=0;



                double predited_scores_copy[sizeof(predited_scores)/sizeof(predited_scores[0])];
                for (i = 0; i < sizeof(predited_scores)/sizeof(predited_scores[0]); i++) {
                     predited_scores_copy[i] = predited_scores[i];   
                 }


                bubbleSortDouble(predited_scores, sizeof(predited_scores)/sizeof(predited_scores[0]));



                int argsort[sizeof(predited_scores)/sizeof(predited_scores[0])] ;
                 for (i = 0; i < sizeof(predited_scores)/sizeof(predited_scores[0]); i++) {
                     for (j= 0; j < sizeof(predited_scores)/sizeof(predited_scores[0]); j++) {
                        val=0;
                         if(predited_scores_copy[j] == predited_scores[i]) {
                         for (q= 0; q < sizeof(argsort)/sizeof(argsort[0]); q++){
                         if(argsort[q] == j) val=1;

                          }
                         if (val==0) argsort[i] = j;
                          }
                          else  {

                          }

                 }
                }



                int u = sizeof(argsort)/sizeof(argsort[0]) - 1;
                int argsort2[sizeof(predited_scores)/sizeof(predited_scores[0])];

                for (i = 0 ; i < u+1 ; i++) {
                    argsort2[i] = argsort[u];
                    u--;  
                }



                int argsort_copy[sizeof(argsort2)/sizeof(argsort2[0])];
                for (i = 0; i < sizeof(argsort2)/sizeof(argsort2[0]); i++) {
                     argsort_copy[i] = argsort2[i];   
                 }


                bubbleSort(argsort2, sizeof(argsort2)/sizeof(argsort2[0]));


               int val2=0;

                int rev_argsort[sizeof(argsort2)/sizeof(argsort2[0])];
                 for (i = 0; i < sizeof(argsort2)/sizeof(argsort2[0]); i++) {
                 val2=0;
                     for (j= 0; j < sizeof(argsort2)/sizeof(argsort2[0]); j++) {

                      if(argsort_copy[j] == argsort2[i]) {
                         for (q= 0; q < sizeof(rev_argsort)/sizeof(rev_argsort[0]); q++){
                         if(rev_argsort[q] == j) val2=1;
                          }
                          if (val2==0) rev_argsort[i] = j;
                          }

                }
                }



                int true_scores2[sizeof(predited_scores)/sizeof(predited_scores[0])];
                for (i = 0; i < sizeof(argsort2)/sizeof(argsort2[0]); i++) {
                     true_scores2[i] = true_scores[argsort2[i]];
                 }


                int predited_scores2[sizeof(predited_scores)/sizeof(predited_scores[0])];
                for (i = 0; i < sizeof(argsort2)/sizeof(argsort2[0]); i++) {
                     predited_scores2[i] = predited_scores[argsort2[i]];
                 }




               double tableau[sizeof(argsort2)/sizeof(argsort2[0])][sizeof(argsort2)/sizeof(argsort2[0])] ={0};

                for (i = 0; i < sizeof(tuple1)/sizeof(tuple1[0]); i++) {

                tableau[tuple1[i]][tuple1[i]] = (double)( pow((double) 2,(double) true_scores2[tuple1[i]]) - 1) / log2((double) tuple1[i] + 2);
                tableau[tuple1[i]][tuple2[i]] = (double)( pow((double) 2,(double) true_scores2[tuple1[i]]) - 1) / log2((double) tuple2[i] + 2);
                tableau[tuple2[i]][tuple2[i]] = (double)( pow((double) 2,(double) true_scores2[tuple2[i]]) - 1) / log2((double) tuple2[i] + 2);
                tableau[tuple2[i]][tuple1[i]] = (double)( pow((double) 2,(double) true_scores2[tuple2[i]]) - 1) / log2((double) tuple1[i] + 2);

              }

                double z_ndcg = 0;
                double rho = 0;
                double rho_complement = 0;
                double lambda_val = 0;
                double lambdas[sizeof(true_scores2)/sizeof(true_scores2[0])]={0};




                for (i = 0; i < sizeof(tuple1)/sizeof(tuple1[0]); i++) {
                        double insideexp=predited_scores2[tuple1[i]] - predited_scores2[tuple2[i]];
                        z_ndcg = (double) abs(tableau[tuple1[i]][tuple2[i]] - tableau[tuple1[i]][tuple1[i]] + tableau[tuple2[i]][tuple1[i]] - tableau[tuple2[i]][tuple2[i]]) / idcg[0];
                        rho = (double) 1/(double) (1 + exp2(insideexp));
                        rho_complement = 1.0 - rho;
                        lambda_val = (double) z_ndcg * rho;
                        lambdas[tuple1[i]] += lambda_val;
                        lambdas[tuple2[i]] -= lambda_val;


                }



                for (i = 0; i < sizeof(lambdas)/sizeof(lambdas[0]); i++) {
                     lambdas[i] = lambdas[rev_argsort[i]];
                 }






                int count=0;
                int pair=0;

                for (i = 0; i < nbKeys[0]*5*2+6 ; i++) {

                 if (tab[0][i] == query_key[0] ) {
                    for (j = i+4; j < i+3*2+4; j++) {
                        if (pair==0) {
                        lambda[tab[0][j]] = lambdas[count];
                        count+=1;
                        pair=1;
                        }
                        else {
                        pair=0;
                        }
                    }
                  }
                }



                return 0;
              } 
                  __global__ void fill_lambdas(int *maxLengthValues, double *lambda, int *true_scores, double *predited_scores, int *tuple1,int *tuple2,double *idcg,int *query_keys, int tab[][5], int *nbKeys)
                  {
                     compute_lambda(maxLengthValues, &true_scores[threadIdx.x], &predited_scores[threadIdx.x], &tuple1[threadIdx.x], &tuple2[threadIdx.x], &idcg[threadIdx.x], &query_keys[threadIdx.x], tab, nbKeys, lambda);      
                  }         

            """)
            
            
            #On va appeler la fonction fill_lambdas (qui elle même appelle compute_lambda cf explications ci-dessus) en passant en argumant les variables préparées en amont
            fill_lambdas_c = mod.get_function("fill_lambdas")
            fill_lambdas_c(maxLengthValues_gpu, lambda_gpu, true_scores_gpu, pred_scores_gpu, tuple11_gpu, tuple2_gpu, idcg_gpu, query_keys_gpu, tab_gpu, nbKeys_gpu, block=(4,1,1))
            
            
            lambdas = np.ones_like(lambdaFinal)
            # on récupère le lambdas qui nous intéresse
            cuda.memcpy_dtoh(lambdas, lambda_gpu)
        
    
            
            
            # Implémentation de l'arbre scikit-learn
            tree = DecisionTreeRegressor(max_depth=50)
            # On fit l'arbre aux lambdas
            tree.fit(self.training_data[:,2:], lambdas)
            # On ajoute l'arbre à notre liste d'arbres
            self.trees.append(tree)
            # On prédit grâce à notre arbre et à nos features  (la fonction predict est en dessous)
            # Calcule la prédiction pour chaque document grâce à nos précédents arbres et au nouveau
                
            prediction = tree.predict(self.training_data[:,2:])
            # On met à jour nos prédictions à l'aide de notre prédiciton récente et du learning_rate
            predicted_scores += prediction * self.learning_rate

    
    def predict(self, data):
        data = np.array(data)
        query_indexes = group_queries(data, 0)
        predicted_scores = np.zeros(len(data))
        for query in query_indexes:
            results = np.zeros(len(query_indexes[query]))
            for tree in self.trees:
                results += self.learning_rate * tree.predict(data[query_indexes[query], 1:])
            predicted_scores[query_indexes[query]] = results
        return predicted_scores

    #Fonction validate qui prédit sur la base test selon les tous les arbres qu'on a construit, qui calcule aussi la moyenne des NDCG
    def validate(self, data, k):
        data = np.array(data)
        query_indexes = group_queries(data, 1)
        average_ndcg = []
        predicted_scores = np.zeros(len(data))
        for query in query_indexes:
            results = np.zeros(len(query_indexes[query]))
            for tree in self.trees:
                results += self.learning_rate * tree.predict(data[query_indexes[query], 2:])
            predicted_sorted_indexes = np.argsort(results)[::-1]
            t_results = data[query_indexes[query], 0]
            t_results = t_results[predicted_sorted_indexes]
            predicted_scores[query_indexes[query]] = results
            dcg_val = dcg_l(t_results, k)
            idcg_val = ideal_dcg_l(t_results, k)
            ndcg_val = (dcg_val / idcg_val)
            average_ndcg.append(ndcg_val)
        average_ndcg = np.nanmean(average_ndcg)
        return average_ndcg, predicted_scores

  

In [ ]:
import numpy as np
import pandas as pd
import time


# Main fonction
def main():
    start_time = time.time()
    total_ndcg = 0.0
    df=pd.read_csv('train2.csv')
    values=[[df.loc[(df.index == j),][str(u)][j] for u in list(range(1,49))] for j in range(len(df.index))][0:18]
    training_data = np.asarray(values)
    df2=pd.read_csv('test2.csv')
    values2=[[df2.loc[(df2.index == j),][str(u)][j] for u in list(range(1,49))] for j in range(len(df2.index))][0:18]
    test_data = np.asarray(values2)
    # On construit les 100 arbres sur la base test
    model = LambdaMART(training_data, 100, 0.001)
    model.fit()
    # On tronque sur les 10 premiers docs pour les calculs de la moyenne des DCG
    # On calcule les scores prédits sur la base test
    average_ndcg, predicted_scores = model.validate(test_data, 10) 
    print("--- %s seconds ---" % (time.time() - start_time))


    
if __name__ == '__main__':
    main()

** d) Comparaison des résultats, conclusion et ouverture **

<tr>
<td>**Localisation**</td>
<td>**Nombre d'arbres**</td>
<td>**Temps** (s)</td>
</tr>
<tr>
<td>CPU</td>
<td>20</td>
<td>23.1</td>
</tr>
<tr>
<td>GPU</td>
<td>20</td>
<td>15.4</td>
</tr>
<tr>

<tr>
<td>CPU</td>
<td>100</td>
<td>27.44</td>
</tr>
<tr>
<td>GPU</td>
<td>100</td>
<td>23.8</td>
</tr>


Ainsi, le temps nécessaire au déroulement de l'algorithme LambdaMart est plus rapide sur GPU (en parallélisant l'algo) que CPU, que ce soit pour 20 ou 100 arbres, et ce dans notre configuration (MacBook Air processeur i5 vs GPU présenté dans la partie précédente).
L'écart semble cependant se réduire lorsque le nombre d'arbres augmente. De plus, lorsque la taille de la base de données est trop importante, un problème de mémoire dans le GPU bloque l'exécution du code (nous n'avons pas jugé utile de prendre un GPU plus puissant dans ce cas). 

Enfin, nous pensons que l'exécution sur le GPU pourraît être bien plus rapide en optimisant l'ensemble du code que nous avons écrit en C (c'était une première pour chacun de nous deux, et nous nous sommes beaucoup plus concentrés sur la cohérence des types et des pointeurs, sur la parallélisation et sur le simple fonctionnement de l'algorithme global que sur l'optimsiation de chaque brique de code en C).

Nous avons toutefois pu rentrer en détail dans le fonctionnement de l'algorithme LambdaMART, pu configurer et utiliser un GPU en autonomie, adapter la logique de l'algorithme afin de le rendre parallélisable, écrire des fonctions en C exécutées sur le GPU et utiliser pyCuda pour la première fois, et avons ainsi énormément appris à tous les niveaux.
